
### Telco Customer Churn using ANN
#### based on examples from: <br> https://www.kaggle.com/snehithatiger/telco-customer-churn-using-ann 


In [1]:
# linear algebra
import numpy as np

# data processing
import pandas as pd 

#CSV file I/O (e.g. pd.read_csv)
import os

#plotting the graph
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib

# scki-learn library - converting the non-numeric data into numeric data.
from sklearn.preprocessing import LabelEncoder

# scki-learn library - partition the dataset for modelling
from sklearn.model_selection import train_test_split

# scki-learn library - metrics for ml models
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_recall_curve

#scki-learn library - KN Neighbors
from sklearn.neighbors import KNeighborsClassifier

#scki-learn library - Random Tree Models
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# xgboost - used for Tree Classifier Models
from xgboost import XGBClassifier

# scki-learn library - prediction using Naive Bayes Algorithm 
from sklearn.naive_bayes import GaussianNB

# keras - tensorflow for artificial neutral networks
import keras
from keras.models import Sequential
from keras.layers import Dense




Using matplotlib backend: Qt5Agg
Using TensorFlow backend.


In [2]:
# Set the active directory to access the data
os.chdir('/home/mike/Documents/mkp_code/Institute of Data Course/telco-customer-churn-project/data/processed')

In [3]:
#importing the dataset
customer_data = pd.read_csv('Telco-Customer-Churn-Processed.csv')

In [4]:
# Display the first 5 rows
customer_data.head()

,Unnamed: 0,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,0,0,0,1,0,-1.277445,0,1,0,0,...,0,0,0,0,1,2,-1.160323,-0.994971,0,0
1,1,1,0,0,0,0.066327,1,0,0,2,...,0,0,0,1,0,3,-0.259629,-0.173876,0,2
2,2,1,0,0,0,-1.236724,1,0,0,2,...,0,0,0,0,1,3,-0.362660,-0.960399,1,0
3,3,1,0,0,0,0.514251,0,1,0,2,...,2,0,0,1,0,0,-0.746535,-0.195400,0,2
4,4,0,0,0,0,-1.236724,1,0,1,0,...,0,0,0,0,1,2,0.197365,-0.941193,1,0


In [5]:
# What does the data look like?
customer_data.describe()

,Unnamed: 0,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7.043000e+03,7043.000000,7043.000000,7043.000000,7043.000000,...,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7.043000e+03,7.043000e+03,7043.000000,7043.000000
mean,3521.000000,0.504756,0.162147,0.483033,0.299588,-2.219500e-17,0.903166,0.940508,0.872923,0.790004,...,0.797104,0.985376,0.992475,0.690473,0.592219,1.574329,-6.103626e-17,8.020467e-17,0.265370,1.751526
std,2033.283305,0.500013,0.368612,0.499748,0.458110,1.000071e+00,0.295752,0.948554,0.737796,0.859848,...,0.861551,0.885002,0.885091,0.833755,0.491457,1.068104,1.000071e+00,1.000071e+00,0.441561,1.497568
min,0.000000,0.000000,0.000000,0.000000,0.000000,-1.318165e+00,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.545860e+00,-9.998503e-01,0.000000,0.000000
25%,1760.500000,0.000000,0.000000,0.000000,0.000000,-9.516817e-01,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,-9.725399e-01,-8.305558e-01,0.000000,0.000000
50%,3521.000000,1.000000,0.000000,0.000000,0.000000,-1.372744e-01,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,0.000000,1.000000,2.000000,1.857327e-01,-3.897629e-01,0.000000,2.000000
75%,5281.500000,1.000000,0.000000,1.000000,1.000000,9.214551e-01,1.000000,2.000000,1.000000,2.000000,...,2.000000,2.000000,2.000000,1.000000,1.000000,2.000000,8.338335e-01,6.637555e-01,1.000000,3.000000
max,7042.000000,1.000000,1.000000,1.000000,1.000000,1.613701e+00,1.000000,2.000000,2.000000,2.000000,...,2.000000,2.000000,2.000000,2.000000,1.000000,3.000000,1.794352e+00,2.826469e+00,1.000000,4.000000


In [6]:
# checking if any null data exists
customer_data.isnull().sum()

Unnamed: 0          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
tenure_group        0
dtype: int64

In [7]:
# drop a column that is not useful
customer_data = customer_data.drop(columns = ['Unnamed: 0'])


In [8]:
# Check the dataset again
customer_data.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,...,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_group
0,0,0,1,0,-1.277445,0,1,0,0,2,...,0,0,0,0,1,2,-1.160323,-0.994971,0,0
1,1,0,0,0,0.066327,1,0,0,2,0,...,0,0,0,1,0,3,-0.259629,-0.173876,0,2
2,1,0,0,0,-1.236724,1,0,0,2,2,...,0,0,0,0,1,3,-0.362660,-0.960399,1,0
3,1,0,0,0,0.514251,0,1,0,2,0,...,2,0,0,1,0,0,-0.746535,-0.195400,0,2
4,0,0,0,0,-1.236724,1,0,1,0,0,...,0,0,0,0,1,2,0.197365,-0.941193,1,0


In [9]:
# X=Features, y=Target
X = customer_data.iloc[:, 0:19]
y = customer_data.Churn

In [10]:
# train test spliting
# split the data into a training and a test part.
# the models will be trained on the training data set and tested on the test data set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)

In [11]:
#print length of X_train, X_test, y_train, y_test
print ("X_train: ", len(X_train))
print("X_test: ", len(X_test))
print("y_train: ", len(y_train))
print("y_test: ", len(y_test))

X_train:  4930
X_test:  2113
y_train:  4930
y_test:  2113


In [12]:
# Model the K Neighbor
knc = KNeighborsClassifier()
knc.fit(X_train, y_train)
knc.fit(X_train, y_train)
print('Accuracy score of KNN training set: {:.3f}'.format(knc.score(X_train, y_train)))
print('Accuracy score of KNN test set: {:.3f}'.format(knc.score(X_test, y_test)))


Accuracy score of KNN training set: 0.829
Accuracy score of KNN test set: 0.780


In [13]:
# Print Results - KNN Classifier
y_knc = knc.predict(X_test)

print('confusion_matrix of KNN: ', confusion_matrix(y_test, y_knc))
print('precision_score of KNN: ', precision_score(y_test, y_knc))
print('recall_score of KNN: ', recall_score(y_test, y_knc))
print('precision_recall_curve of KNN: ', precision_recall_curve(y_test, y_knc))

confusion_matrix of KNN:  [[1351  196]
 [ 268  298]]
precision_score of KNN:  0.6032388663967612
recall_score of KNN:  0.5265017667844523
precision_recall_curve of KNN:  (array([0.26786559, 0.60323887, 1.        ]), array([1.        , 0.52650177, 0.        ]), array([0, 1]))


In [14]:
# Model the Random Forest Regressor
rfr = RandomForestRegressor()
rfr.fit(X_train, y_train)
print('Accuracy score (training): {:.3f}'.format(rfr.score(X_train, y_train)))
print('Accuracy score (test): {:.3f}'.format(rfr.score(X_test, y_test)))

Accuracy score (training): 0.891
Accuracy score (test): 0.274


In [15]:
# Model Random Forest Classifier
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train)
print('Accuracy score Random Forest Classifier training set: {:.3f}'.format(rfc.score(X_train, y_train)))
print('Accuracy score Random Forest Classifier test set: {:.3f}'.format(rfc.score(X_test, y_test)))

Accuracy score Random Forest Classifier training set: 0.998
Accuracy score Random Forest Classifier test set: 0.796


In [16]:
y_rfc = rfc.predict(X_test)

print('confusion_matrix of Random Forest Classifier: ', confusion_matrix(y_test, y_rfc))
print('precision_score of Random Forest Classifier: ', precision_score(y_test, y_rfc))
print('recall_score of Random Forest Classifier: ', recall_score(y_test, y_rfc))
print('precision_recall_curve of Random Forest Classifier: ', precision_recall_curve(y_test, y_rfc))

confusion_matrix of Random Forest Classifier:  [[1414  133]
 [ 297  269]]
precision_score of Random Forest Classifier:  0.6691542288557214
recall_score of Random Forest Classifier:  0.4752650176678445
precision_recall_curve of Random Forest Classifier:  (array([0.26786559, 0.66915423, 1.        ]), array([1.        , 0.47526502, 0.        ]), array([0, 1]))


In [17]:
# Model Decision Tree Classifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 42)
classifier.fit(X_train, y_train)

print('accuracy of Decision Tree Classifier training set: {:.3f}'.format(classifier.score(X_train,y_train)))
print('accuaracy of Decision Tree Classifier test set: {:.3f}'.format(classifier.score(X_test, y_test)))

accuracy of Decision Tree Classifier training set: 0.998
accuaracy of Decision Tree Classifier test set: 0.734


In [18]:
y_dtc = classifier.predict(X_test)
print('accuracy_score of decision tree classifier: ', accuracy_score(y_dtc, y_test))
print('confusion_matrix of decision tree classifier: ', confusion_matrix(y_dtc, y_test))
print('precision_score of decision tree classifier: ', precision_score(y_dtc, y_test))
print('recall_score of decision tree classifier: ', recall_score(y_dtc, y_test))
print('precision_recall_curve of decision tree classifier: ', precision_recall_curve(y_dtc, y_test))

accuracy_score of decision tree classifier:  0.7335541883577852
confusion_matrix of decision tree classifier:  [[1277  293]
 [ 270  273]]
precision_score of decision tree classifier:  0.4823321554770318
recall_score of decision tree classifier:  0.5027624309392266
precision_recall_curve of decision tree classifier:  (array([0.2569806 , 0.48233216, 1.        ]), array([1.        , 0.50276243, 0.        ]), array([0, 1]))


In [19]:
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
print('Accuracy XGBclassifier on train set: {:.3f}'.format(xgb.score(X_train, y_train)))
print('Accuracy XGBClassifier on test set: {:.3f}'.format(xgb.score(X_test, y_test)))

Accuracy XGBclassifier on train set: 0.950
Accuracy XGBClassifier on test set: 0.796


In [20]:
y_xgbc = xgb.predict(X_test)

# predicting Confusion matrix, accuracy score,precision score, recall score
print('accuracy_score of xgboost: ', accuracy_score(y_test, y_xgbc))
print('confusion_matrix of xgboost: ', confusion_matrix(y_test, y_xgbc))
print('precision_score of xgboost: ', precision_score(y_test, y_xgbc))
print('recall_score of xgboost: ', recall_score(y_test, y_xgbc))
print('precision_recall_curve of xgboost: ', precision_recall_curve(y_test, y_xgbc))

accuracy_score of xgboost:  0.79649787032655
confusion_matrix of xgboost:  [[1408  139]
 [ 291  275]]
precision_score of xgboost:  0.6642512077294686
recall_score of xgboost:  0.48586572438162545
precision_recall_curve of xgboost:  (array([0.26786559, 0.66425121, 1.        ]), array([1.        , 0.48586572, 0.        ]), array([0, 1]))


In [21]:
# prediction using Naive Bayes Algorithm 
nbc = GaussianNB()
nbc.fit(X_train, y_train)

print('accuracy of Naive Bayes training set: {:.3f}'.format(nbc.score(X_train,y_train)))
print('accuaracy of Naive Bayes test set: {:.3f}'.format(nbc.score(X_test, y_test)))

accuracy of Naive Bayes training set: 0.748
accuaracy of Naive Bayes test set: 0.769


In [22]:
y_nb = nbc.predict(X_test)

print('accuracy_score of Naive Bayes: ', accuracy_score(y_test, y_nb))
print('confusion_matrix of Naive Bayes: ', confusion_matrix(y_test, y_nb))
print('precision_score of Naive Bayes: ', precision_score(y_test, y_nb))
print('recall_score of Naive Bayes: ', recall_score(y_test, y_nb))
print('precision_recall_curve of Naive Bayes: ', precision_recall_curve(y_test, y_nb))

accuracy_score of Naive Bayes:  0.7685754850922859
confusion_matrix of Naive Bayes:  [[1204  343]
 [ 146  420]]
precision_score of Naive Bayes:  0.5504587155963303
recall_score of Naive Bayes:  0.7420494699646644
precision_recall_curve of Naive Bayes:  (array([0.26786559, 0.55045872, 1.        ]), array([1.        , 0.74204947, 0.        ]), array([0, 1]))


In [23]:
# Initialising the Multi Layer Perceptron (MLP)
classifier = Sequential()

In [24]:
# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = 19))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [25]:
# Adding the second hidden layer
classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))

In [26]:
# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

In [27]:
# Compiling the MLP
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
# Fitting the MLP to the Training set
classifier.fit(X_train, y_train, batch_size = 5, epochs = 100)

Train on 4930 samples
Epoch 1/100
4930/4930 [==============================] - 2s 313us/sample - loss: 0.4677 - acc: 0.7698
Epoch 2/100
4930/4930 [==============================] - 1s 285us/sample - loss: 0.4295 - acc: 0.7895
Epoch 3/100
4930/4930 [==============================] - 1s 290us/sample - loss: 0.4241 - acc: 0.7919
Epoch 4/100
4930/4930 [==============================] - 1s 298us/sample - loss: 0.4226 - acc: 0.7963
Epoch 5/100
4930/4930 [==============================] - 1s 288us/sample - loss: 0.4192 - acc: 0.8028
Epoch 6/100
4930/4930 [==============================] - 1s 294us/sample - loss: 0.4190 - acc: 0.8006
Epoch 7/100
4930/4930 [==============================] - 2s 305us/sample - loss: 0.4180 - acc: 0.8012
Epoch 8/100
4930/4930 [==============================] - 1s 294us/sample - loss: 0.4161 - acc: 0.8032
Epoch 9/100
4930/4930 [==============================] - 1s 303us/sample - loss: 0.4161 - acc: 0.8010
Epoch 10/100
4930/4930 [==============================] - 1s

In [32]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                320       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 17        
Total params: 609
Trainable params: 609
Non-trainable params: 0
_________________________________________________________________


In [36]:
# prediction using Neural Networks 
y_ann =classifier.predict_proba(X_test)

# predicting Confusion matrix, accuracy score,precision score, recall score
print('accuracy_score of xgboost: ', accuracy_score(y_test, y_ann))
print('confusion_matrix of xgboost: ', confusion_matrix(y_test, y_ann))
print('precision_score of xgboost: ', precision_score(y_test, y_ann))
print('recall_score of xgboost: ', recall_score(y_test, y_ann))
print('precision_recall_curve of xgboost: ', precision_recall_curve(y_test, y_ann))

ValueError: Classification metrics can't handle a mix of binary and continuous targets